VAR : Vector Auto-Regression (벡터자기회귀모형)

종속변수의 과거에만 영향 : 자기회귀

종속변수와 독립 변수의 과거에 영향 : 벡터자기회귀

1. 데이터 전처리 : 결측지, 이상치, 중복값 확인

2. 요약통계량 확인 : summary statistics

3. 시계열들의 stationary 확인 : 시계열의 평균, 분산 등이 시간과 관계없이 일정

4. 변수들 간의 correlation 확인

5. VAR 분석

6. Granger Causality Test

7. 충격반응모형

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.api import VAR

In [ ]:
data = pd.read_csv('data.csv', encoding = 'cp949').fillna(0)
data.index = data['날짜']
data = data.drop('날짜', axis = 1)
data = data.drop('PM2.5', axis = 1)

In [ ]:
plt.figure(figsize = (15, 8))
plt.plot(data['PM10'])
plt.show()

In [ ]:
quantile_25 = np.quantile(data['PM10'], 0.25)
quantile_75 = np.quantile(data['PM10'], 0.75)
IQR = quantile_75 - quantile_25
min = quantile_25 - 1.5 * IQR
max = quantile_75 + 1.5 * IQR
data1 = data[(data['PM10'] >= min) & (data['PM10'] <= max)]

In [ ]:
train = data1.iloc[:-5, :]
test = data1.iloc[-5:, :]

In [ ]:
plt.figure(figsize = (15, 8))
plt.plot(train['PM10'])
plt.show()

In [ ]:
adf_test = adfuller(train['PM10'], autolag = 'AIC')
print('ADF Test')
print(f'ADF Statistics : {adf_test[0]}')
print(f'p-value : {adf_test[1]}')
print('Critical values :')
for key, value in adf_test[4].items():
  print(f'\t{key} : {value}')
# p-value < 0.05 이므로 H0 기각
# Stationary 하다.

In [ ]:
kpss_test = kpss(train['PM10'])
print('KPSS Test')
print(f'KPSS Statistics : {kpss_test[0]}')
print(f'p-value : {kpss_test[1]}')
print('Critical values :')
for key, value in kpss_test[3].items():
  print(f'\t{key} : {value}')
# p-value < 0.05 이므로 H0 기각
# Stationary 하다.

In [ ]:
var_model = VAR(train)
p = []
for i in range(1, 10):
  result = var_model.fit(i)
  p.append(result.aic)
plt.plot(list(range(1, 10)), p)
plt.show()

In [ ]:
var_model_final = var_model.fit(4)
var_model_final.summary()

In [ ]:
forecast = pd.DataFrame(var_model_final.forecast(y = train.values, steps = 5), index = test.index, columns = ['PM10', '오존', '이산화질소', '일산화탄소', '아황산가스', '평균기온', '강수량', '풍속', '풍향', '상대습도'])
result = pd.concat([forecast['PM10'], test['PM10']], axis = 1)
result.columns = ['예측', '실제']
print(result)

plt.figure(figsize = (15, 8))
plt.plot(result['예측'], label = 'Forecast')
plt.plot(result['실제'], label = 'Real')
plt.legend()
plt.show()